# <font color = "#81263f" size = "60">11. Scripting with LLMs</font>

## <font style="background:#81263f;color:white">11.1 Exercise: Cutting Polyline</font>

### <font color = "#81263f">11.1.1 Concepts</font>

### <font color = "#81263f">11.1.2 An Existing Function</font>

In [ ]:
def draw_polyline(canvas,vertices, line_width = 20, color = '#81263f'):
    canvas.line_width = line_width
    canvas.line_cap = 'square'
    canvas.begin_path()
    for i in range(len(vertices)):
        if i==0:
            canvas.move_to(vertices[i][0], vertices[i][1])
        else:
            canvas.line_to(vertices[i][0], vertices[i][1])
    canvas.stroke_style = color
    canvas.stroke()

### <font color = "#81263f">11.1.3 Existing Conditions</font>

In [ ]:
from ipycanvas import Canvas
canvas = Canvas(width=650, height=350)
verts = [[50,40], [150,45], [200,90], [250,290], [50,290], [50,40]]

draw_polyline(canvas, verts)
canvas

### <font color = "#81263f">11.1.4 ChatGPT</font>

### <font color = "#81263f">11.1.5 Point on Polyline</font>

In [ ]:
def compute_distance(p1, p2):
    return ((p2[0] - p1[0])**2 + (p2[1] - p1[1])**2)**0.5

def point_on_polyline(vertices, t):
    total_length = sum(compute_distance(vertices[i], vertices[i+1]) for i in range(len(vertices)-1))
    target_distance = t * total_length

    accumulated_length = 0.0

    for i in range(len(vertices)-1):
        segment_length = compute_distance(vertices[i], vertices[i+1])
        
        if accumulated_length + segment_length >= target_distance:
            fraction = (target_distance - accumulated_length) / segment_length
            x = vertices[i][0] + fraction * (vertices[i+1][0] - vertices[i][0])
            y = vertices[i][1] + fraction * (vertices[i+1][1] - vertices[i][1])
            
            return [x, y], i  # Return point and segment index
        
        accumulated_length += segment_length
    return None, None

# Test
vertices = [[50,40], [150,45], [200,90], [250,290], [50,290], [50,40]]
t = 0.35
print(point_on_polyline(vertices, t))

### <font color = "#81263f">11.1.6 Draw Point</font>

In [ ]:
def draw_point(canvas, x, y, thickness=2, color='black'):
    # Set the color for the point
    canvas.fill_style = color
    
    # Draw the point
    canvas.fill_arc(x, y, thickness / 2, 0, 2 * 3.141592653589793)  # The last two arguments define the start and end angles for a full circle

In [ ]:
pt, idx = point_on_polyline(vertices, t)
draw_point(canvas, pt[0], pt[1], thickness=20, color='black')
canvas

### <font color = "#81263f">11.1.7 Cut Polyline 1</font>

In [ ]:
def cut_polyline(vertices, t1, t2):
    if t1 > t2:
        t1, t2 = t2, t1  # Swap if t1 is greater than t2

    point1, index1 = point_on_polyline(vertices, t1)
    point2, index2 = point_on_polyline(vertices, t2)
    print(index1, index2)
    # Construct the new polyline
    new_polyline = []

    # Add the starting segment
    if index1 == index2:
        new_polyline.extend([point1, point2])
    else:
        new_polyline.append(point1)
        for i in range(index1 + 1, index2):
            new_polyline.append(vertices[i])
        new_polyline.append(point2)

    return new_polyline

canvas.clear()
new_polyline = cut_polyline(vertices, 0.2, 0.4)
draw_polyline(canvas, new_polyline)
canvas

### <font color = "#81263f">11.1.8 Cut Polyline 2</font>

In [ ]:
def cut_polyline(vertices, t1, t2):
    if t1 > t2:
        t1, t2 = t2, t1  # Swap if t1 is greater than t2

    point1, index1 = point_on_polyline(vertices, t1)
    point2, index2 = point_on_polyline(vertices, t2)

    # Construct the new polyline
    new_polyline = []

    # Add vertices up to t1
    new_polyline.extend(vertices[:index1])
    new_polyline.append(point1)
    
    # Add point from t2 and then remaining vertices
    new_polyline.append(point2)
    new_polyline.extend(vertices[index2+1:])

    return new_polyline

# Test
canvas.clear()
new_polyline = cut_polyline(vertices, 0.2, 0.4)
draw_polyline(canvas, new_polyline)
canvas

### <font color = "#81263f">11.1.9 Cut Polyline 3</font>

In [ ]:
def cut_polyline(vertices, t1, t2):
    if t1 > t2:
        t1, t2 = t2, t1  # Swap if t1 is greater than t2

    point1, index1 = point_on_polyline(vertices, t1)
    point2, index2 = point_on_polyline(vertices, t2)

    # Construct the new polyline
    new_polyline = []

    # If both points are on the same segment, just create a segment between them
    if index1 == index2:
        return [point1, point2]

    # Add vertices from t2 to the end
    new_polyline.append(point2)
    new_polyline.extend(vertices[index2+1:])
    
    # Close loop to start and continue up to t1
    new_polyline.extend(vertices[:index1])
    new_polyline.append(point1)

    return new_polyline

canvas.clear()
new_polyline = cut_polyline(vertices, 0.2, 0.4)
draw_polyline(canvas, new_polyline)
canvas

### <font color = "#81263f">11.1.10 Cut Polyline 4</font>

In [ ]:
from ipycanvas import Canvas
canvas = Canvas(width=650, height=350)

verts0 = [[600,10], [630,50], [610,250], [640,130], [620,5], [600,10]]
verts1 = [[10,10], [100,120], [150,80], [210,30], [260,5], [10,10]]
verts2 = [[10,310], [100,290], [150,320], [210,130], [260,315], [10,310]]
verts3 = [[300,10], [370,50], [420,20], [480,30], [520,5], [300,10]]
verts4 = [[300,310], [370,290], [420,320], [480,230], [520,315], [300,310]]
polyline = [verts0, verts1, verts2, verts3, verts4]

for poly in polyline:
    draw_polyline(canvas, cut_polyline(poly, 0.4, 0.5), line_width=2)
canvas

### <font color = "#81263f">11.1.11 Random Polyline Cutting in a Grid 1</font>

In [ ]:
import random
from ipycanvas import Canvas

def perturb_vertex(vertex, bounds, max_offset=10): # max_offset = 1/3 of 30
    x, y = vertex
    x_min, y_min, x_max, y_max = bounds

    dx = random.uniform(-max_offset, max_offset)
    dy = random.uniform(-max_offset, max_offset)

    # Ensure the perturbed vertex does not move outside its cell
    x = min(max(x + dx, x_min), x_max)
    y = min(max(y + dy, y_min), y_max)

    return [x, y]

def generate_grid(rows=10, cols=15, cell_size=30):
    grid = []
    for i in range(rows):
        row = []
        for j in range(cols):
            top_left_x = j * cell_size
            top_left_y = i * cell_size
            
            top_left = [top_left_x, top_left_y]
            top_right = [top_left_x + cell_size, top_left_y]
            bottom_left = [top_left_x, top_left_y + cell_size]
            bottom_right = [top_left_x + cell_size, top_left_y + cell_size]
            
            bounds = (top_left_x, top_left_y, top_right[0], bottom_left[1])

            cell = [
                perturb_vertex(top_left, bounds),
                perturb_vertex(top_right, bounds),
                perturb_vertex(bottom_right, bounds),
                perturb_vertex(bottom_left, bounds),
                perturb_vertex(top_left, bounds)  # Close the loop
            ]
            row.append(cell)
        grid.append(row)
    return grid

def draw_grid(grid, canvas, color='black'):
    for row in grid:
        for quad in row:
            draw_polyline(canvas, quad, 2)


In [ ]:
# Test
canvas = Canvas(width=650, height=350)  # Based on 20x15 grid with cell_size=30
grid = generate_grid()
draw_grid(grid, canvas)
canvas

### <font color = "#81263f">11.1.12 Random Polyline Cutting in a Grid 2</font>

In [ ]:
import random
from ipycanvas import Canvas

def perturb_vertex(vertex, bounds, max_offset=10): # max_offset = 1/3 of 30
    x, y = vertex
    x_min, y_min, x_max, y_max = bounds

    dx = random.uniform(-max_offset, max_offset)
    dy = random.uniform(-max_offset, max_offset)

    # Ensure the perturbed vertex does not move outside its cell
    x = min(max(x + dx, x_min), x_max)
    y = min(max(y + dy, y_min), y_max)

    return [x, y]

def generate_grid(rows=10, cols=20, cell_size=30):
    grid = []
    for i in range(rows):
        row = []
        for j in range(cols):
            top_left_x = j * cell_size
            top_left_y = i * cell_size
            
            top_left = [top_left_x, top_left_y]
            top_right = [top_left_x + cell_size, top_left_y]
            bottom_left = [top_left_x, top_left_y + cell_size]
            bottom_right = [top_left_x + cell_size, top_left_y + cell_size]
            
            bounds = (top_left_x, top_left_y, top_right[0], bottom_left[1])

            # Perturb each vertex
            perturbed_top_left = perturb_vertex(top_left, bounds)
            perturbed_top_right = perturb_vertex(top_right, bounds)
            perturbed_bottom_right = perturb_vertex(bottom_right, bounds)
            perturbed_bottom_left = perturb_vertex(bottom_left, bounds)

            # Construct the closed quad
            quad = [perturbed_top_left, perturbed_top_right, perturbed_bottom_right, perturbed_bottom_left, perturbed_top_left]
            row.append(quad)
        grid.append(row)
    return grid

# ... [Drawing functions remain unchanged]

def draw_grid(grid, canvas, color='black'):
    for row in grid:
        for quad in row:
            draw_polyline(canvas, quad, 2)


In [ ]:
# Test
canvas = Canvas(width=650, height=350)  # Based on 20x15 grid with cell_size=30
grid = generate_grid()
draw_grid(grid, canvas)
canvas

### <font color = "#81263f">11.1.13 Random Polyline Cutting in a Grid 3</font>

In [ ]:
# ... [Other functions and definitions remain unchanged from previous examples]

def generate_and_cut_grid(rows=10, cols=20, cell_size=30):
    grid = generate_grid(rows, cols, cell_size)
    
    for i in range(rows):
        for j in range(cols):
            t1 = random.uniform(0.1, 0.5)  # We ensure t2 > t1 by limiting t1's upper bound
            t2 = random.uniform(0.55, 0.9)
            grid[i][j] = cut_polyline(grid[i][j], t1, t2)
    
    return grid



In [ ]:
# Test
canvas = Canvas(width=650, height=350)  # Based on 20x15 grid with cell_size=30
grid = generate_and_cut_grid()
draw_grid(grid, canvas)
canvas

## <font style="background:#81263f;color:white">11.2 Exercise: Simple Cell Growth</font>

### <font color = "#81263f">11.2.1 Concepts</font>

### <font color = "#81263f">11.2.2 Simple Cell Growth 1</font>

In [ ]:
import random
from ipycanvas import Canvas, hold_canvas
from ipywidgets import AppLayout, Button#, display

# Constants
GRID_ROWS = 5
GRID_COLS = 10
CELL_SIZE = 50
WIDTH = GRID_COLS * CELL_SIZE
HEIGHT = GRID_ROWS * CELL_SIZE
CELL_COLORS = {0: 'white', 1: '#81263f', 2: '#344f4f', 3: '#263266'}

# Initialize grid with zeros
grid = [[0 for _ in range(GRID_COLS)] for _ in range(GRID_ROWS)]

# Randomly initialize three cells
initial_cells = random.sample([(x, y) for x in range(GRID_ROWS) for y in range(GRID_COLS)], 3)
for i, (x, y) in enumerate(initial_cells, 1):
    grid[x][y] = i

# Canvas setup
canvas = Canvas(width=WIDTH, height=HEIGHT)

# Drawing function
def draw_grid():
    with hold_canvas(canvas):
        canvas.clear()
        for row in range(GRID_ROWS):
            for col in range(GRID_COLS):
                cell_value = grid[row][col]
                canvas.fill_style = CELL_COLORS[cell_value]
                canvas.fill_rect(col * CELL_SIZE, row * CELL_SIZE, CELL_SIZE, CELL_SIZE)
                canvas.stroke_style = 'black'
                canvas.stroke_rect(col * CELL_SIZE, row * CELL_SIZE, CELL_SIZE, CELL_SIZE)

# Cell growth function
def grow_cells():
    new_grid = [row.copy() for row in grid]
    directions = [(0, 1), (1, 0), (0, -1), (-1, 0)]
    
    for x in range(GRID_ROWS):
        for y in range(GRID_COLS):
            if grid[x][y] != 0:
                possible_cells = []
                for dx, dy in directions:
                    nx, ny = x + dx, y + dy
                    if 0 <= nx < GRID_ROWS and 0 <= ny < GRID_COLS and grid[nx][ny] == 0 and new_grid[nx][ny] == 0:
                        possible_cells.append((nx, ny))
                if possible_cells:
                    nx, ny = random.choice(possible_cells)
                    new_grid[nx][ny] = grid[x][y]
                    
    return new_grid

# Event function
def on_run_clicked(_):
    while any(0 in row for row in grid):  # while there are empty cells
        new_grid = grow_cells()
        if new_grid == grid:  # If no growth is possible
            break
        grid[:] = new_grid
        draw_grid()

# UI setup
run_button = Button(description="Run")
run_button.on_click(on_run_clicked)

app_layout = AppLayout(center=canvas,
                       footer=run_button,
                       pane_heights=[0, HEIGHT, '60px'])

draw_grid()
display(app_layout)


### <font color = "#81263f">11.2.3 Simple Cell Growth 2</font>

In [ ]:
import random
import time
from IPython.display import clear_output
from ipycanvas import Canvas, hold_canvas
from ipywidgets import AppLayout, Button, HBox#, display

# Constants
GRID_ROWS = 5
GRID_COLS = 10
CELL_SIZE = 50
WIDTH = GRID_COLS * CELL_SIZE
HEIGHT = GRID_ROWS * CELL_SIZE
CELL_COLORS = {0: 'white', 1: '#81263f', 2: '#344f4f', 3: '#263266'} # custom colors are applied

# Initialize grid
grid = [[0 for _ in range(GRID_COLS)] for _ in range(GRID_ROWS)]

def initialize_cells():
    global grid
    grid = [[0 for _ in range(GRID_COLS)] for _ in range(GRID_ROWS)]
    initial_cells = random.sample([(x, y) for x in range(GRID_ROWS) for y in range(GRID_COLS)], 3)
    for i, (x, y) in enumerate(initial_cells, 1):
        grid[x][y] = i

initialize_cells()

# Canvas setup
canvas = Canvas(width=WIDTH, height=HEIGHT)

# Drawing function
def draw_grid():
    with hold_canvas(canvas):
        canvas.clear()
        for row in range(GRID_ROWS):
            for col in range(GRID_COLS):
                cell_value = grid[row][col]
                canvas.fill_style = CELL_COLORS[cell_value]
                canvas.fill_rect(col * CELL_SIZE, row * CELL_SIZE, CELL_SIZE, CELL_SIZE)
                canvas.stroke_style = 'black'
                canvas.stroke_rect(col * CELL_SIZE, row * CELL_SIZE, CELL_SIZE, CELL_SIZE)

# Cell growth function
def grow_cells():
    new_grid = [row.copy() for row in grid]
    directions = [(0, 1), (1, 0), (0, -1), (-1, 0)]
    
    for x in range(GRID_ROWS):
        for y in range(GRID_COLS):
            if grid[x][y] != 0:
                possible_cells = []
                for dx, dy in directions:
                    nx, ny = x + dx, y + dy
                    if 0 <= nx < GRID_ROWS and 0 <= ny < GRID_COLS and grid[nx][ny] == 0 and new_grid[nx][ny] == 0:
                        possible_cells.append((nx, ny))
                if possible_cells:
                    nx, ny = random.choice(possible_cells)
                    new_grid[nx][ny] = grid[x][y]
                    
    return new_grid

# Event functions
def on_run_clicked(_):
    while any(0 in row for row in grid):  # while there are empty cells
        new_grid = grow_cells()
        if new_grid == grid:  # If no growth is possible
            break
        grid[:] = new_grid
        draw_grid()
        time.sleep(0.1)
        clear_output(wait=True)
        display(app_layout)

def on_reset_clicked(_):
    initialize_cells()
    draw_grid()

# UI setup
run_button = Button(description="Run")
run_button.on_click(on_run_clicked)

reset_button = Button(description="Reset")
reset_button.on_click(on_reset_clicked)

controls = HBox([run_button, reset_button])

app_layout = AppLayout(center=canvas,
                       footer=controls,
                       pane_heights=[0, HEIGHT, '60px'])

draw_grid()
display(app_layout)


## <font style="background:#81263f;color:white">11.3 Exercise: Gradient Descent / Ascent</font>

### <font color = "#81263f">11.3.1 Concepts</font>

### <font color = "#81263f">11.3.2 Gradient Descent</font>

In [ ]:
from ipywidgets import Button, AppLayout
from ipycanvas import MultiCanvas
import numpy as np
import random

canvas = MultiCanvas(2, width=650, height=400)
background_layer = canvas[0]
interaction_layer = canvas[1]


In [ ]:
WIDTH = 650
HEIGHT = 400
X_RANGE = [-2, 4]
Y_RANGE = [-10, 10]
LR = 0.01
EPSILON = 0.01

def f(x):
    return x**5 - 6*x**4 + 5*x**3 + 10*x**2 - 6*x + 2

def to_canvas_coordinates(x, y):
    canvas_x = (x - X_RANGE[0]) / (X_RANGE[1] - X_RANGE[0]) * WIDTH
    canvas_y = HEIGHT - (y - Y_RANGE[0]) / (Y_RANGE[1] - Y_RANGE[0]) * HEIGHT
    return canvas_x, canvas_y

def draw_curve():
    x_vals = np.linspace(X_RANGE[0], X_RANGE[1], 400)
    y_vals = f(x_vals)
    coords = [to_canvas_coordinates(x, y) for x, y in zip(x_vals, y_vals)]
    
    background_layer.stroke_style = "black"
    background_layer.begin_path()
    background_layer.move_to(*coords[0])
    for x, y in coords:
        background_layer.line_to(x, y)
    background_layer.stroke()

draw_curve()


In [ ]:
x_point = random.uniform(X_RANGE[0], X_RANGE[1])

def gradient_descent():
    global x_point
    prev_x = float('inf')
    
    while abs(x_point - prev_x) > EPSILON:
        prev_x = x_point
        
        right_gradient = (f(x_point + EPSILON) - f(x_point)) / EPSILON
        left_gradient = (f(x_point) - f(x_point - EPSILON)) / EPSILON
        
        # Draw the current position in black
        x_canvas, y_canvas = to_canvas_coordinates(x_point, f(x_point))
        interaction_layer.fill_style = '#344f4f'
        interaction_layer.fill_arc(x_canvas, y_canvas, 5, 0, 2 * np.pi)
        
        if abs(left_gradient) < abs(right_gradient):
            x_point -= LR * abs(left_gradient)
        else:
            x_point += LR * abs(right_gradient)
    
    # Draw the final position in red
    x_canvas, y_canvas = to_canvas_coordinates(x_point, f(x_point))
    interaction_layer.fill_style = 'red'
    interaction_layer.fill_arc(x_canvas, y_canvas, 5, 0, 2 * np.pi)

In [ ]:
def on_run_click(b):
    gradient_descent()

def on_reset_click(b):
    global x_point
    x_point = random.uniform(X_RANGE[0]+.5, X_RANGE[1]-1.5)   # changed the range for convenience
    interaction_layer.clear()
    x_canvas, y_canvas = to_canvas_coordinates(x_point, f(x_point))
    interaction_layer.fill_style = 'red'
    interaction_layer.fill_arc(x_canvas, y_canvas, 5, 0, 2 * np.pi)

run_button = Button(description="Run", layout={'width': '100px'})
reset_button = Button(description="Reset", layout={'width': '100px'})

run_button.on_click(on_run_click)
reset_button.on_click(on_reset_click)


In [ ]:
AppLayout(center=canvas, footer=run_button, header=reset_button)